code credit goes to:
https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lemcm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lemcm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lemcm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Importing Gensim
import gensim
from gensim import corpora

In [4]:
df = pd.read_csv("Cleaned_df.csv", index_col = "Unnamed: 0")

In [5]:
df = df.loc[:,["Summary", "Text"]]

In [6]:
lemmatizer = WordNetLemmatizer()

In [7]:
stop_words = set(stopwords.words("english"))

In [8]:
def word_tokens(row):
    return word_tokenize(row)

In [9]:
def clean_data(df):
    df = df.apply(word_tokenize)
    df = df.map(lambda x: [i.lower() for i in x if i.lower() not in stop_words])
    df = df.map(lambda x: [lemmatizer.lemmatize(i) for i in x if i.lower() if i.isalnum()])
    df = df.map(lambda x: ' '.join(x))

    return df

In [10]:
def LDA(df, n_topics):
    df = clean_data(df)
    
    #preprocessing
    doc_clean = [i.split() for i in df]
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    
    #Build model:
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(doc_term_matrix, num_topics=n_topics, id2word = dictionary, passes=50)
    
    my_dict = {'Topic_' + str(i): [token for token, score in ldamodel.show_topic(i, topn=10)] for i in range(0, ldamodel.num_topics)}
    
    #print results
    return my_dict

In [14]:
tester_df = df.sample(200).reset_index(drop=True)

In [15]:
LDA(tester_df["Text"],5)

{'Topic_0': ['br',
  'taste',
  'get',
  'butter',
  'chocolate',
  'one',
  'good',
  'peanut',
  'brand',
  'even'],
 'Topic_1': ['good',
  'product',
  'bar',
  'box',
  'get',
  'would',
  'br',
  'find',
  'like',
  'flavor'],
 'Topic_2': ['tea',
  'br',
  'like',
  'one',
  'taste',
  'good',
  'treat',
  'food',
  'would',
  'flavor'],
 'Topic_3': ['taste',
  'coffee',
  'great',
  'food',
  'good',
  'like',
  'one',
  'product',
  'br',
  'love'],
 'Topic_4': ['br',
  'tea',
  'make',
  'like',
  'one',
  'taste',
  'flavor',
  'great',
  'made',
  'use']}